In [47]:
import pandas as pd
import os
import json
from IPython.display import display

Step1: DataFrame for each air pollutant
- read each csv
- input into dataframe
- convert into long format

In [48]:
nevarne_snovi = [("PM10", 1999), ("SO2", 1997)]
meseci = ["Jan", "Feb", "Mar", "Apr", "Maj","Jun", "Jul", "Avg", "Sep", "Okt", "Nov","Dec"]


In [49]:
pd_dict = {}


for snov,zac_leto in nevarne_snovi:
    dir_path = f"podatki/{snov}"

    for leto, leto_file in enumerate(sorted(os.listdir(dir_path)),start=zac_leto):
        file_path = os.path.join(dir_path, leto_file)
        df = pd.read_csv(file_path, na_values=["", ",", " ","/"])
        
        df.insert(1, "Leto", leto)
        # get rid of two postaja

        df = df.melt(id_vars=["Postaja", "Leto"], value_vars=meseci,var_name="Mesec",value_name=snov)
        if snov not in pd_dict:
            pd_dict[snov] = df
        else:
            pd_dict[snov] = pd.concat([pd_dict[snov], df])
    
    pd_dict[snov]['Postaja'] = pd_dict[snov]['Postaja'].str.lower()

    pd_dict[snov].to_csv(f"podatki/df/{snov}.csv")

    print(f"Reading complete!")

Reading complete!
Reading complete!


In [ ]:
""" pm_dict = pd_dict['PM10']

pm_dict[pm_dict['Postaja'] == 'žerjav'] """

,Postaja,Leto,Mesec,PM10
13,žerjav,2010,Jan,64.0
35,žerjav,2010,Feb,44.0
57,žerjav,2010,Mar,25.0
79,žerjav,2010,Apr,19.0
101,žerjav,2010,Maj,15.0
...,...,...,...,...
288,žerjav,2023,Avg,37.0
326,žerjav,2023,Sep,24.0
364,žerjav,2023,Okt,21.0
402,žerjav,2023,Nov,20.0


Step2: Map stations to regions

In [ ]:
map_dict = json.load(open("podatki/slovar.json"))
map_dict = {key: [v.lower() for v in values] for key,values in map_dict.items()}

""" vsi_mappingsi = set([p for postaje in map_dict.values() for p in postaje])
vse_postaje = set([postaja for df in pd_dict.values() for postaja in df['Postaja'].unique()])

vse_postaje - vsi_mappingsi """

tmp = pd.Series(map_dict).explode()



InvalidIndexError: Reindexing only valid with uniquely valued Index objects